# Kymograph maker
This notebook will find competitive cellular "events" in the simplest definition (i.e. a loser cell apoptosis) and return information about the spatiotemporal distrubition of counterpart competitive events (i.e. winner cell mitosis) in the form of a kymograph.

kymograph:
/ˈkʌɪmə(ʊ)ɡrɑːf/
_noun_ 
a graphical representation of spatial positions (of wild-type cells or mitoses) over time in which a spatial axis represents time, focused around an apoptotic event.

Contents:

- Load modules
- Set experiment data path
- Load tracking data
- Isolate tracks of interest (target track)
- Iterate over tracks, generating and saving kymographs

To-do:

- [x] Rewrite this to iterate over many target tracks
- [x] Insert filter for short scr_tracks
- [ ] Think about saving out as something other than .npy

In [ ]:
import napari
import btrack
import numpy as np
from skimage.io import imread
import os
print("Napari version no.:", napari.__version__)
print("btrack version no.:", btrack.__version__)
from btrack.utils import import_HDF, import_JSON, tracks_to_napari
from tqdm.notebook import tnrange, tqdm
import matplotlib.pyplot as plt

def find_apoptosis_time(target_track, index): ### if index is set to True then the index of the apoptotic time (wrt target_track) is returned
    for i, j in enumerate(target_track.label):
        if j == 'APOPTOSIS' and target_track.label[i+1] == 'APOPTOSIS' and target_track.label[i+2] == 'APOPTOSIS': # and target_track.label[i+3] =='APOPTOSIS' and target_track.label[i+4] =='APOPTOSIS':
            apop_index = i
            break
    apop_time = target_track.t[apop_index]
    if index == True: 
        return apop_index
    else: 
        return apop_time

### Set experiment data path 

In [ ]:
# print("Input experiment number")
# experiment_no = input()
# root_path = os.path.join('/home/nathan/data/kraken/h2b/giulia/', experiment_no)
root_path = '/home/nathan/data/kraken/h2b/giulia/GV0807'  ## this overwrites input option for ease 
# gfp_path = os.path.join(root_path, 'Pos3/stacks/gfp.tif')
# rfp_path = os.path.join(root_path, 'Pos3/stacks/rfp.tif')
# bf_path = os.path.join(root_path, 'Pos3/stacks/bf.tif')
tracks_path = os.path.join(root_path, 'Pos3/Pos3_aligned/HDF/segmented.hdf5')

### Load tracking data

In [ ]:
with btrack.dataio.HDF5FileHandler(tracks_path, 'r', obj_type = "obj_type_1") as hdf:
    wt_tracks = hdf.tracks
with btrack.dataio.HDF5FileHandler(tracks_path, 'r', obj_type = "obj_type_2") as hdf:
    scr_tracks = hdf.tracks
print("Tracks loaded")

In [ ]:
scr_apops = [scr_track for scr_track in scr_tracks if scr_track.fate.name == 'APOPTOSIS']

### Set kymograph parameters

In [ ]:
radius = 400
delta_t = 400
num_radial_bins = 20
radial_bin = radius / num_radial_bins
num_temporal_bins = 20
temporal_bin = delta_t / num_temporal_bins
print("size of temporal and radial bins:",temporal_bin,",", radial_bin)

### Calculate kymographs as npy arrays

In [ ]:
max_N_wt = 100
for scr_track in tqdm(scr_apops):
    if len(scr_track) <= 10: ### arbitrary length of scr track needed
        print("Scr_ID", scr_track.ID, "skipped")
        continue
    if scr_track.ID in [1, 3, 4, 6, 10, 11, 16, 17, 18, 29, 47, 50, 54, 56, 58, 65]: ### scr-tracks already analysed
        print("Scr_ID", scr_track.ID, "skipped")
        continue    
    print("Scr_ID:", scr_track.ID)
    target_track = scr_track 
    apop_index = find_apoptosis_time(target_track, index = True)
    apop_time = find_apoptosis_time(target_track, index = False) 
    ## instead of going over all tracks, could go over all tracks that are definitely in_frame (using func) or in zone 
    ## set empty data variables to store 
    num_wt, num_wt_mito = np.zeros((num_radial_bins, num_temporal_bins)), np.zeros((num_radial_bins, num_temporal_bins))
    wt_IDs, wt_mito_IDs = np.zeros((num_radial_bins, num_temporal_bins, max_N_wt)), np.zeros((num_radial_bins, num_temporal_bins, max_N_wt))
    ## load one wt_track
    for j, wt_track in enumerate(tqdm(wt_tracks)):
        print("ID:", wt_track.ID)
        ## load first distance/radial bin
        for n in range(num_radial_bins):
            #print("radial", range(int(radial_bin * n), int(radial_bin * (n+1))))
            ## load first temporal bin, over negative and positive range
            for l, m in enumerate(range(int(-num_temporal_bins/2), int(num_temporal_bins/2))): ## l introduced to iterate through pos integers for data storage
                #print("time",range(int(apop_time+temporal_bin*m),int(apop_time+temporal_bin*(m+1))))
                ## load first timepoint of wt_track
                for i in range(len(wt_track)): 
                    ## if FIRST time point is in temporal bin (centered around apop_time)
                    if wt_track.t[i] in (range(int(apop_time+temporal_bin*m),int(apop_time+temporal_bin*(m+1)))):
                        ## calculate if within euclidean distance: if min of radial distance bin sqrd < (wt_track.x (at first time point) - target_track.x (at apop_time)) both squared + corresponding for yboth squared < max_dist squared
                        if ((radial_bin * n)**2) <= (wt_track.x[i] - target_track.x[apop_index])**2 + (wt_track.y[i] - target_track.y[apop_index])**2 < ((radial_bin * (n+1))**2):

                            num_wt[n,l] += 1 #str(wt_track.ID)
                            print("wt_track", wt_track.ID)
                            print("radial", range(int(radial_bin * n), int(radial_bin * (n+1))))
                            print("time",range(int(apop_time+temporal_bin*m),int(apop_time+temporal_bin*(m+1))))

                            ## storing ID in 3D ndarray
                            for p in range(len(wt_IDs[n,l,:])):
                                if wt_IDs[n,l,p] == 0:          ### if WT_ID hasnt been stored in array at p YET, store there by overwriting 0/empty entry
                                    wt_IDs[n,l,p] = wt_track.ID
                                    break                        

                            #print("WT ID:", wt_track.ID, "time:", wt_track.t[i], "radial bin:", (n*radial_bin),"-", ((n+1)*radial_bin), "temporal bin:", (temporal_bin * m),"-", (temporal_bin * (m+1)),"i,m,n:", i,m,n)
                            if wt_track.fate.name == "DIVIDE" and "'METAPHASE', 'METAPHASE'," in str(wt_track.label[int(apop_time+(temporal_bin*m))-wt_track.t[0]:int(apop_time+(temporal_bin*(m+1)))-wt_track.t[0]]): ##explainer: this double condition states that if wt_track ends in mitosis but also has THREE (?) sequential metaphase classifications within the time window of that temporal bin then the condition is met
                                num_wt_mito[n,l] += 1 #str(wt_track.ID)
                                print("wt_track mitosis", wt_track.ID)
                                print("radial", range(int(radial_bin * n), int(radial_bin * (n+1))))
                                print("time",range(int(apop_time+temporal_bin*m),int(apop_time+temporal_bin*(m+1))))

                                ## storing ID in 3D ndarray
                                for p in range(len(wt_mito_IDs[n,l,:])):
                                    if wt_mito_IDs[n,l,p] == 0:
                                        wt_mito_IDs[n,l,p] = wt_track.ID
                                        break   


                                #print("MITO ID:", wt\_track.ID, "time:", wt_track.t[i], "radial bin:", (n*radial_bin),"-", ((n+1)*radial_bin), "temporal bin:", (temporal_bin * m),"-", (temporal_bin * (m+1)),"i,m,n:", i,m,n)
                            break ## if track is in bin, break loop iterating over all track positions to skip to next bin ##CHECK THIS

    scr_ID_fn = 'Scr_'+str(target_track.ID)
    fn = os.path.join(root_path, 'analysis/raw_numbers/', scr_ID_fn)

    num_wt_fn = fn + '_num_wt'
    num_wt_mito_fn = fn + '_num_wt_mito'
    wt_IDs_fn = fn + '_wt_IDs'
    wt_mito_IDs_fn = fn + '_wt_mito_IDs'

    np.save(num_wt_fn, num_wt)
    np.save(num_wt_mito_fn, num_wt_mito)
    np.save(wt_IDs_fn, wt_IDs)
    np.save(wt_mito_IDs_fn, wt_mito_IDs)